In [1]:
# GPU计算
!nvidia-smi

Fri Apr 13 20:46:38 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   39C    P5     9W / 140W |    432MiB /  6078MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
import pip
for pkg in ['mxnet', 'mxnet-cu75', 'mxnet-cu80', 'mxnet-cu90']:
    pip.main(['show', pkg])

Name: mxnet-cu90
Version: 1.2.0b20180413
Summary: MXNet is an ultra-scalable deep learning framework. This version uses CUDA-9.0.
Home-page: https://github.com/apache/incubator-mxnet
Author: UNKNOWN
Author-email: UNKNOWN
License: Apache 2.0
Location: /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages
Requires: requests, numpy, graphviz


In [2]:
import mxnet as mx
[mx.cpu(), mx.gpu(), mx.gpu(1)]

/home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[cpu(0), gpu(0), gpu(1)]

In [3]:
# NDArray的GPU计算
from mxnet import nd
x = nd.array([1,2,3])
x.context

cpu(0)

In [13]:
# GPU上创建内存
a = nd.array([1,2,3], ctx=mx.gpu())
b = nd.zeros((3,2), ctx=mx.gpu())
c = nd.random.uniform(shape=(2,3), ctx=mx.gpu())
print(nd.dot(a,b))
(a,b,c)


[0. 0.]
<NDArray 2 @gpu(0)>


(
 [1. 2. 3.]
 <NDArray 3 @gpu(0)>, 
 [[0. 0.]
  [0. 0.]
  [0. 0.]]
 <NDArray 3x2 @gpu(0)>, 
 [[0.6369917  0.12847    0.17249882]
  [0.9368206  0.59183455 0.94970965]]
 <NDArray 2x3 @gpu(0)>)

In [9]:
import sys
try:
    nd.array([1,2,3], ctx=mx.gpu(10))
except mx.MXNetError as err:
    sys.stderr.write(str(err))


[10:36:25] src/storage/storage.cc:65: Check failed: e == cudaSuccess || e == cudaErrorCudartUnloading CUDA: invalid device ordinal

Stack trace returned 10 entries:
[bt] (0) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x3032da) [0x7fd34f4862da]
[bt] (1) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x303901) [0x7fd34f486901]
[bt] (2) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x28e9b9f) [0x7fd351a6cb9f]
[bt] (3) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x28f08f8) [0x7fd351a738f8]
[bt] (4) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x23f2516) [0x7fd351575516]
[bt] (5) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(MXNDArrayCreateEx+0x158) [0x7fd351575d88]
[bt] (6) /home/ly/anaconda3/envs/learning/lib/python3.6/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7fd3ab487ec

In [16]:
y = x.copyto(mx.gpu())
z = x.as_in_context(mx.gpu())
(y, z)

(
 [1. 2. 3.]
 <NDArray 3 @gpu(0)>, 
 [1. 2. 3.]
 <NDArray 3 @gpu(0)>)

In [17]:
yy = y.as_in_context(mx.gpu())
zz = z.copyto(mx.gpu())
(yy is y, zz is z)

(True, False)

In [18]:
# GPU上的计算
nd.exp(z+2)*y


[ 20.085537 109.1963   445.2395  ]
<NDArray 3 @gpu(0)>

In [20]:
try:
    x+y
except mx.MXNetError as err:
    sys.stderr.write(str(err))

[10:39:24] src/imperative/./imperative_utils.h:55: Check failed: inputs[i]->ctx().dev_mask() == ctx.dev_mask() (2 vs. 1) Operator broadcast_add require all inputs live on the same context. But the first argument is on cpu(0) while the 2-th argument is on gpu(0)

Stack trace returned 10 entries:
[bt] (0) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x3032da) [0x7fd34f4862da]
[bt] (1) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x303901) [0x7fd34f486901]
[bt] (2) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x24e392d) [0x7fd35166692d]
[bt] (3) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x24ea569) [0x7fd35166d569]
[bt] (4) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x241c2a9) [0x7fd35159f2a9]
[bt] (5) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(MXImperativeInvokeEx+0x6f) [0x7fd

In [25]:
# 默认会复制回CPU的操作
print(y)
print(y.asnumpy())
print(y.sum().asscalar())
print(y.sum())


[1. 2. 3.]
<NDArray 3 @gpu(0)>
[1. 2. 3.]
6.0

[6.]
<NDArray 1 @gpu(0)>


In [27]:
# Gluon的GPU计算
from mxnet import gluon
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(1))
net.initialize(ctx=mx.gpu())
print(net)

Sequential(
  (0): Dense(None -> 1, linear)
)


In [30]:
data = nd.random.uniform(shape=[3,2], ctx=mx.gpu())
net(data)


[[0.01246436]
 [0.00535382]
 [0.00620071]]
<NDArray 3x1 @gpu(0)>

In [31]:
net[0].weight.data()


[[0.0068339  0.01299825]]
<NDArray 1x2 @gpu(0)>